In [ ]:
try:
    import google.colab
    print("🚀 Running in Google Colab - all packages pre-installed!")
    IN_COLAB = True
except:
    print("💻 Running locally")
    IN_COLAB = False
    # Uncomment below if packages missing:
    # !pip install pandas numpy scikit-learn matplotlib seaborn

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import warnings
warnings.filterwarnings('ignore')

# LOAD AND EXPLORE DATA
print("=== CALIFORNIA HOUSING PRICE PREDICTION ===\n")

# Loading the California housing dataset
housing = fetch_california_housing()
X = pd.DataFrame(housing.data, columns=housing.feature_names)
y = housing.target  # Prices in hundreds of thousands of dollars

print("✅ Dataset loaded successfully!")
print(f"Dataset Shape: {X.shape}")
print(f"Features: {list(X.columns)}")
print(f"Target: House prices (in $100k)")

# EXPLORATORY DATA ANALYSIS
print("\n=== EXPLORATORY DATA ANALYSIS ===")
print(f"Missing values: {X.isnull().sum().sum()}")
print(f"Target statistics:")
print(f"- Mean price: ${y.mean():.1f}00k")
print(f"- Median price: ${np.median(y):.1f}00k")
print(f"- Price range: ${y.min():.1f}00k - ${y.max():.1f}00k")

print(f"\nFeature statistics:")
for col in X.columns:
    print(f"- {col}: {X[col].mean():.2f} ± {X[col].std():.2f}")

# DATA PREPROCESSING
print("\n=== DATA PREPROCESSING ===")

print(f"Features shape: {X.shape}")
print(f"Target shape: {y.shape}")

# Splitting the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(f"Training set: {X_train.shape}")
print(f"Test set: {X_test.shape}")

# Scaling the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
print("✅ Features scaled using StandardScaler")


print(f"Target (y) statistics: Mean={y.mean():.2f}, Std={y.std():.2f}")

# IMPLEMENT SVR (SUPPORT VECTOR REGRESSION)
print("\n=== SVR IMPLEMENTATION ===")

# Basic SVR with RBF kernel
svr_basic = SVR(kernel='rbf')
svr_basic.fit(X_train_scaled, y_train)
svr_pred = svr_basic.predict(X_test_scaled)

# Calculating metrics
svr_mse = mean_squared_error(y_test, svr_pred)
svr_rmse = np.sqrt(svr_mse)
svr_r2 = r2_score(y_test, svr_pred)
svr_mae = mean_absolute_error(y_test, svr_pred)

print(f"SVR Basic Results:")
print(f"- RMSE: ${svr_rmse:.2f}00k")
print(f"- R²: {svr_r2:.4f}")
print(f"- MAE: ${svr_mae:.2f}00k")

# SVR with hyperparameter tuning
print("\n🔧 Tuning SVR hyperparameters...")
svr_param_grid = {
    'C': [0.1, 1, 10, 100],
    'kernel': ['rbf', 'linear'],
    'gamma': ['scale', 'auto', 0.001, 0.01],
    'epsilon': [0.01, 0.1, 0.2]
}

# Using smaller grid for faster execution
svr_param_grid_fast = {
    'C': [1, 10, 100],
    'kernel': ['rbf', 'linear'],
    'gamma': ['scale', 'auto']
}

svr_grid = GridSearchCV(SVR(), svr_param_grid_fast, cv=3, scoring='neg_mean_squared_error', n_jobs=-1)
svr_grid.fit(X_train_scaled, y_train)
svr_best = svr_grid.best_estimator_

svr_best_pred = svr_best.predict(X_test_scaled)
svr_best_mse = mean_squared_error(y_test, svr_best_pred)
svr_best_rmse = np.sqrt(svr_best_mse)
svr_best_r2 = r2_score(y_test, svr_best_pred)
svr_best_mae = mean_absolute_error(y_test, svr_best_pred)

print(f"SVR Best Parameters: {svr_grid.best_params_}")
print(f"SVR Best Results:")
print(f"- RMSE: ${svr_best_rmse:.2f}00k")
print(f"- R²: {svr_best_r2:.4f}")
print(f"- MAE: ${svr_best_mae:.2f}00k")

# IMPLEMENT KNN REGRESSION
print("\n=== KNN REGRESSION IMPLEMENTATION ===")

# Testing different K values
k_values = range(1, 21)
knn_rmse_scores = []

print("Testing different K values...")
for k in k_values:
    knn = KNeighborsRegressor(n_neighbors=k)
    cv_scores = cross_val_score(knn, X_train_scaled, y_train, cv=3,
                               scoring='neg_mean_squared_error')
    rmse_score = np.sqrt(-cv_scores.mean())  # Convert back to RMSE
    knn_rmse_scores.append(rmse_score)
    if k <= 10 or k % 5 == 0:  # Print progress
        print(f"K={k}: RMSE={rmse_score:.3f}")

# Finding best K
best_k_idx = np.argmin(knn_rmse_scores)
best_k = k_values[best_k_idx]
best_k_rmse = knn_rmse_scores[best_k_idx]

print(f"\n🎯 Best K value: {best_k} (CV RMSE: {best_k_rmse:.3f})")

# Training KNN with best K
knn_best = KNeighborsRegressor(n_neighbors=best_k)
knn_best.fit(X_train_scaled, y_train)
knn_pred = knn_best.predict(X_test_scaled)

# Calculating metrics
knn_mse = mean_squared_error(y_test, knn_pred)
knn_rmse = np.sqrt(knn_mse)
knn_r2 = r2_score(y_test, knn_pred)
knn_mae = mean_absolute_error(y_test, knn_pred)

print(f"KNN Best Results:")
print(f"- RMSE: ${knn_rmse:.2f}00k")
print(f"- R²: {knn_r2:.4f}")
print(f"- MAE: ${knn_mae:.2f}00k")

# COMPARING MODELS
print("\n=== MODEL COMPARISON ===")

results = {
    'SVR Basic': {'RMSE': svr_rmse, 'R²': svr_r2, 'MAE': svr_mae},
    'SVR Tuned': {'RMSE': svr_best_rmse, 'R²': svr_best_r2, 'MAE': svr_best_mae},
    'KNN': {'RMSE': knn_rmse, 'R²': knn_r2, 'MAE': knn_mae}
}

print("Model Performance Summary:")
print("=" * 50)
for model, metrics in results.items():
    print(f"{model:12} | RMSE: ${metrics['RMSE']:.2f}00k | R²: {metrics['R²']:.4f} | MAE: ${metrics['MAE']:.2f}00k")

# Winner based on RMSE (lower is better)
rmse_values = {model: metrics['RMSE'] for model, metrics in results.items()}
best_model = min(rmse_values, key=rmse_values.get)
best_rmse = rmse_values[best_model]

print(f"\n🏆 Best Model: {best_model}")
print(f"   Best RMSE: ${best_rmse:.2f}00k")
print(f"   This means predictions are typically off by ~${best_rmse*100:.0f}k")

# DETAILED ANALYSIS
print(f"\n=== DETAILED ANALYSIS ===")

# Choosing best model for detailed analysis
if best_model == 'SVR Tuned':
    final_pred = svr_best_pred
    final_model = svr_best
    model_type = "SVR"
elif best_model == 'KNN':
    final_pred = knn_pred
    final_model = knn_best
    model_type = "KNN"
else:
    final_pred = svr_pred
    final_model = svr_basic
    model_type = "SVR"

print(f"Analysis for best model: {best_model}")

# Prediction vs Actual analysis
residuals = y_test - final_pred
print(f"Residual Analysis:")
print(f"- Mean residual: ${np.mean(residuals):.3f}00k")
print(f"- Std of residuals: ${np.std(residuals):.3f}00k")

# Percentage errors
percentage_errors = np.abs(residuals / y_test) * 100
print(f"Prediction Accuracy:")
print(f"- Mean Absolute Percentage Error: {np.mean(percentage_errors):.1f}%")
print(f"- 90% of predictions within: ±${np.percentile(np.abs(residuals), 90):.2f}00k")